# Keras API with Callbacks

### Import the required modules

In [128]:
import tensorflow as tf
from reader import AudioReader
from preprocessing import Padding, Normalization
from preprocessing import MFCC

### Define the Hyperparameters

In [129]:
PREPROCESSING_ARGS = {
    'sampling_rate': 16000,
    'frame_length_in_s': 0.032,
    'frame_step_in_s': 0.016,
    'num_mel_bins': 21,
    'lower_frequency': 40,
    'upper_frequency': 5000,
    'num_coefficients': 14
}

TRAINING_ARGS = {
    'batch_size': 20,
    'learning_rate': 0.01,
    'end_learning_rate': 1.e-5,
    'epochs': 20,
    'width_multiplier': 0.4,
    'begin_step_%': 0.1,
    'end_step_%': 0.9,
    'initial_sparsity': 0.10,
    'final_sparsity': 0.82
}

### Create train/val/test Datasets

In [130]:
train_ds = tf.data.Dataset.list_files(['/tmp/msc-train/*down*', '/tmp/msc-train/*up*'])
val_ds = tf.data.Dataset.list_files(['/tmp/msc-val/*down*', '/tmp/msc-val/*up*'])
test_ds = tf.data.Dataset.list_files(['/tmp/msc-test/*down*', '/tmp/msc-test/*up*'])

### Define the Data Pipeline

In [131]:
!ls /tmp/msc-train/* | head -n5
!ls /tmp/msc-val/* | head -n5
!ls /tmp/msc-test/* | head -n5

/tmp/msc-train/down_004ae714_nohash_0.wav
/tmp/msc-train/down_00b01445_nohash_1.wav
/tmp/msc-train/down_00f0204f_nohash_0.wav
/tmp/msc-train/down_0132a06d_nohash_1.wav
/tmp/msc-train/down_0137b3f4_nohash_2.wav
ls: write error: Broken pipe
/tmp/msc-val/down_0132a06d_nohash_4.wav
/tmp/msc-val/down_063d48cf_nohash_0.wav
/tmp/msc-val/down_0a9f9af7_nohash_0.wav
/tmp/msc-val/down_0ff728b5_nohash_4.wav
/tmp/msc-val/down_14872d06_nohash_0.wav
ls: write error: Broken pipe
/tmp/msc-test/down_099d52ad_nohash_2.wav
/tmp/msc-test/down_0ff728b5_nohash_3.wav
/tmp/msc-test/down_10ace7eb_nohash_3.wav
/tmp/msc-test/down_14587ff0_nohash_0.wav
/tmp/msc-test/down_19e246ad_nohash_0.wav
ls: write error: Broken pipe


### Read a batch of data

In [132]:
audio_reader = AudioReader(tf.int16)
padding = Padding(PREPROCESSING_ARGS['sampling_rate'])
normalization = Normalization(tf.int16)
#mel_spec_processor = MelSpectrogram(**PREPROCESSING_ARGS)
mfcc_processor = MFCC(**PREPROCESSING_ARGS)

LABELS = ['down', 'up']

def prepare_for_training(feature, label):
    feature = tf.expand_dims(feature, -1)
    label_id = tf.argmax(label == LABELS)

    return feature, label_id

train_ds = (train_ds
            .map(audio_reader.get_audio_and_label)
            .map(padding.pad)
            .map(normalization.normalize)
            .map(mfcc_processor.get_mfccs_and_label)
            .map(prepare_for_training)
            .batch(TRAINING_ARGS['batch_size'])
            .cache())
val_ds = (val_ds
            .map(audio_reader.get_audio_and_label)
            .map(padding.pad)
            .map(normalization.normalize)
            .map(mfcc_processor.get_mfccs_and_label)
            .map(prepare_for_training)
            .batch(TRAINING_ARGS['batch_size']))
test_ds = (test_ds
            .map(audio_reader.get_audio_and_label)
            .map(padding.pad)
            .map(normalization.normalize)
            .map(mfcc_processor.get_mfccs_and_label)
            .map(prepare_for_training)
            .batch(TRAINING_ARGS['batch_size']))

In [133]:
for example_batch, example_labels in train_ds.take(1):
  print('Batch Shape:', example_batch.shape)
  print('Data Shape:', example_batch.shape[1:])
  print('Labels:', example_labels)

Batch Shape: (20, 61, 14, 1)
Data Shape: (61, 14, 1)
Labels: tf.Tensor([1 1 0 0 0 0 0 1 1 1 0 0 0 1 0 0 1 1 0 1], shape=(20,), dtype=int64)
2024-12-09 15:03:08.683062: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


### Create the Model

sequential convolutional NN

In [134]:
wm = TRAINING_ARGS['width_multiplier']

model = tf.keras.Sequential([ ##width multiplier and model for pruning nelle funz successive 
    tf.keras.layers.Input(shape=example_batch.shape[1:]),
    tf.keras.layers.Conv2D(filters=int(256 * wm), kernel_size=[3, 3], strides=[2, 2],
        use_bias=False, padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1], 
        use_bias=False, padding='same'),
    tf.keras.layers.Conv2D(filters=int(256 * wm), kernel_size=[1, 1], strides=[1, 1],   
       use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.Conv2D(filters=int(256 * wm), kernel_size=[1, 1], strides=[1, 1],   
       use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=len(LABELS)),
    tf.keras.layers.Softmax()
])

### Setup for pruning

In [135]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

begin_step = int(len(train_ds) * TRAINING_ARGS['epochs'] * TRAINING_ARGS['begin_step_%'])
end_step = int(len(train_ds) * TRAINING_ARGS['epochs'] * TRAINING_ARGS['end_step_%'])

pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=TRAINING_ARGS['initial_sparsity'],
        final_sparsity=TRAINING_ARGS['final_sparsity'],
        begin_step=begin_step,
        end_step=end_step
    )
}

#wm = TRAINING_ARGS['width_multiplier']

model_for_pruning = prune_low_magnitude(model, **pruning_params)

In [30]:
model_for_pruning.summary() 

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d  (None, 30, 9, 102)        1838      
 _3 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_batch_  (None, 30, 9, 102)        409       
 normalization_3 (PruneLowM                                      
 agnitude)                                                       
                                                                 
 prune_low_magnitude_re_lu_  (None, 30, 9, 102)        1         
 3 (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_depthw  (None, 30, 9, 102)        919       
 ise_conv2d_2 (PruneLowMagn                                      
 itude)                                               

### Create callbacks

In [136]:
# Learning Rate scheduler
linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=TRAINING_ARGS['learning_rate'],
    end_learning_rate=TRAINING_ARGS['end_learning_rate'],
    decay_steps=len(train_ds) * TRAINING_ARGS['epochs'],
)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(linear_decay)

# Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1,
    mode='auto'
)

### Train the Model (with Callbacks)

- Modify the `learning_rate` argument of the `Adam` optimizer.
- Modify the `fit` method, specifying the `callbacks` argument.

In [137]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = tf.optimizers.Adam(learning_rate=linear_decay)
metrics = [tf.metrics.SparseCategoricalAccuracy()]
model_for_pruning.compile(loss=loss, optimizer=optimizer, metrics=metrics)

update_pruning_step = tfmot.sparsity.keras.UpdatePruningStep()

history = model_for_pruning.fit(
    train_ds, 
    epochs=TRAINING_ARGS['epochs'], 
    validation_data=val_ds, 
    callbacks=[lr_scheduler, early_stopping, update_pruning_step]
)

Epoch 1/20
80/80 [==============================] - 12s 112ms/step - loss: 0.4559 - sparse_categorical_accuracy: 0.7931 - val_loss: 0.5974 - val_sparse_categorical_accuracy: 0.7350 - lr: 0.0095
Epoch 2/20
80/80 [==============================] - 5s 66ms/step - loss: 0.2876 - sparse_categorical_accuracy: 0.8956 - val_loss: 1.6708 - val_sparse_categorical_accuracy: 0.6200 - lr: 0.0090
Epoch 3/20
80/80 [==============================] - 5s 65ms/step - loss: 0.2089 - sparse_categorical_accuracy: 0.9362 - val_loss: 0.3793 - val_sparse_categorical_accuracy: 0.8600 - lr: 0.0085
Epoch 4/20
80/80 [==============================] - 5s 67ms/step - loss: 0.1334 - sparse_categorical_accuracy: 0.9538 - val_loss: 0.2412 - val_sparse_categorical_accuracy: 0.9250 - lr: 0.0080
Epoch 5/20
80/80 [==============================] - 5s 67ms/step - loss: 0.0969 - sparse_categorical_accuracy: 0.9625 - val_loss: 0.2452 - val_sparse_categorical_accuracy: 0.9250 - lr: 0.0075
Epoch 6/20
80/80 [====================

### Show the History

In [33]:
history.history

{'loss': [0.45441532135009766,
  0.26019325852394104,
  0.15863880515098572,
  0.0992484837770462,
  0.08901619166135788,
  0.07165495306253433,
  0.06607170403003693,
  0.057738833129405975,
  0.05761871486902237,
  0.04707843437790871,
  0.04561035335063934,
  0.044971369206905365,
  0.04408722370862961],
 'sparse_categorical_accuracy': [0.8006250262260437,
  0.9006249904632568,
  0.9456250071525574,
  0.9599999785423279,
  0.9624999761581421,
  0.9768750071525574,
  0.9750000238418579,
  0.981249988079071,
  0.9806249737739563,
  0.9887499809265137,
  0.9868749976158142,
  0.9868749976158142,
  0.9906250238418579],
 'val_loss': [0.342609167098999,
  0.14617864787578583,
  0.10345589369535446,
  0.12607049942016602,
  0.13870616257190704,
  0.09479240328073502,
  0.15088890492916107,
  0.07318870723247528,
  0.1285814344882965,
  0.11188601702451706,
  0.11795833706855774,
  0.09999417513608932,
  0.12296637892723083],
 'val_sparse_categorical_accuracy': [0.8650000095367432,
  0.9200

### Save the Model

In [138]:
import os
from time import time

timestamp = int(time())

saved_model_dir = f'./saved_models/{timestamp}'
if not os.path.exists(saved_model_dir):
    os.makedirs(saved_model_dir)
model_for_pruning.save(saved_model_dir)

INFO:tensorflow:Assets written to: ./saved_models/1733756761/assets
INFO:tensorflow:Assets written to: ./saved_models/1733756761/assets


### TFLite Conversion

In [35]:
!ls saved_models

1733485351  1733496861	1733498417  1733500854	1733501673  1733502983
1733495575  1733497584	1733499278  1733501129	1733502148  1733506242
1733495848  1733498015	1733500342  1733501442	1733502504  1733506643


In [139]:
MODEL_NAME = 1733756761

In [140]:
converter = tf.lite.TFLiteConverter.from_saved_model(f'./saved_models/{MODEL_NAME}')
tflite_model = converter.convert()

2024-12-09 15:06:31.745082: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-12-09 15:06:31.745525: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-12-09 15:06:31.878865: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: ./saved_models/1733756761
2024-12-09 15:06:32.041516: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2024-12-09 15:06:32.041661: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: ./saved_models/1733756761
2024-12-09 15:06:32.062484: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-12-09 15:06:32.472914: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: ./saved_models/1733756761
2024-12-09 15:06:32.518488: I tensorflow/cc/saved_model/loader.cc:314] SavedModel load for tags { serve }; Status

In [141]:
tflite_models_dir = './tflite_models'
if not os.path.exists(tflite_models_dir):
    os.makedirs(tflite_models_dir)

In [142]:
tflite_model_name = os.path.join(tflite_models_dir, f'{MODEL_NAME}.tflite')
tflite_model_name

'./tflite_models/1733756761.tflite'

In [143]:
with open(tflite_model_name, 'wb') as fp:
    fp.write(tflite_model)

In [41]:
!ls tflite_models

1733495575.tflite      1733498417.tflite.zip  1733501673.tflite
1733495575.tflite.zip  1733499278.tflite      1733501673.tflite.zip
1733495848.tflite      1733499278.tflite.zip  1733502148.tflite
1733495848.tflite.zip  1733500342.tflite      1733502148.tflite.zip
1733496861.tflite      1733500342.tflite.zip  1733502504.tflite
1733496861.tflite.zip  1733500854.tflite      1733502504.tflite.zip
1733497584.tflite      1733500854.tflite.zip  1733502983.tflite
1733497584.tflite.zip  1733501129.tflite      1733502983.tflite.zip
1733498015.tflite      1733501129.tflite.zip  1733506242.tflite
1733498015.tflite.zip  1733501442.tflite      1733506242.tflite.zip
1733498417.tflite      1733501442.tflite.zip  1733506643.tflite


In [144]:
import zipfile

with zipfile.ZipFile(f'{tflite_model_name}.zip', 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(tflite_model_name)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8a9d9526-dc21-42d6-ba37-8f708634743d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>